In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import SimpleRNN, LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [2]:
dataset = pd.read_csv('data/train.csv')
dataset.head()
dataset.shape

(20800, 5)

In [3]:
nan_indexes = dataset.loc[pd.isna(dataset["text"]), :].index
dataset = dataset.drop(nan_indexes)
dataset.shape

(20761, 5)

In [4]:
print(142 in nan_indexes)

True


In [5]:
X = dataset.text
y = dataset.label

In [6]:
X[143]

'  U.N. Secretary General Complains That The ‘Masses Have Rejected Globalism’ In Favor Of Nationalism Antonio Guterres, elected in October to take over as U.N. secretary general next year, told a conference in his native Lisbon that this trend had undermined the willingness to receive refugees in Europe this year. He said the world must re-establish international protection for refugees coming from war zones such as Syria, but it would not be easy as developed countries were turning to nationalist agendas.   22, 2016 The incoming head of the United Nations warned on Tuesday that ‘losers of globalization’ in rich countries have felt ignored by establishment politicians, prompting them to turn to nationalist agendas, as in the U.S. election and Brexit referendum. “Therefore wait ye upon me, saith the LORD, until the day that I rise up to the prey: for my determination is to gather the nations, that I may assemble the kingdoms, to pour upon them mine indignation, even all my fierce anger:

In [7]:
mean_article_length = 0

for i in range(20761):
    if i in nan_indexes:
        continue
    mean_article_length += len(X[i].split(' '))

mean_article_length /= 20761

mean_article_length

773.6980395934685

In [8]:
max_words = 2000
max_len = 1000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [9]:
def LSTM_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [11]:


model = LSTM_network()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 1000, 50)          100000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_3 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [12]:
model.fit(sequences_matrix,y,batch_size=128,epochs=20,
          validation_split=0.2)


Train on 16608 samples, validate on 4153 samples
Epoch 1/20
16608/16608 [==============================] - 59s 4ms/step - loss: 0.3428 - acc: 0.8525 - val_loss: 0.1678 - val_acc: 0.9429
Epoch 2/20
16608/16608 [==============================] - 58s 3ms/step - loss: 0.1564 - acc: 0.9461 - val_loss: 0.1852 - val_acc: 0.9364
Epoch 3/20
16608/16608 [==============================] - 58s 3ms/step - loss: 0.1217 - acc: 0.9585 - val_loss: 0.1574 - val_acc: 0.9487
Epoch 4/20
16608/16608 [==============================] - 58s 3ms/step - loss: 0.1391 - acc: 0.9535 - val_loss: 0.1793 - val_acc: 0.9372
Epoch 5/20
16608/16608 [==============================] - 58s 3ms/step - loss: 0.1008 - acc: 0.9683 - val_loss: 0.1689 - val_acc: 0.9410
Epoch 6/20
16608/16608 [==============================] - 62s 4ms/step - loss: 0.1229 - acc: 0.9563 - val_loss: 0.2019 - val_acc: 0.9343
Epoch 7/20
16608/16608 [==============================] - 64s 4ms/step - loss: 0.1262 - acc: 0.9551 - val_loss: 0.1676 - val_acc:

In [21]:
def RNN_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = SimpleRNN(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [22]:
model = RNN_network()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
model.fit(sequences_matrix,y,batch_size=128,epochs=20,
          validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 1000, 50)          100000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 64)                7360      
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_7 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________